In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
import statsmodels.api as sm
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download() #<---Use this command to download WordNet in corpora

In [ ]:
train = pd.read_csv("/media/fexunexa/Tree/Kaggle/Git Quora/data/train.csv")

In [ ]:
# https://www.kaggle.com/cstahl12/titanic/titanic-with-keras

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils

print('Keras using {} backend'.format(keras.backend.backend()))  #https://keras.io/backend/

batch_size = np.round(len(quora_train_features_tf)*0.10)
num_classes = 2
epochs = 50

quora_train_features_tf_reshape = quora_train_features_tf.astype('float32')
quora_test_features_tf_reshape = quora_test_features_tf.astype('float32')

print(quora_train_features_tf_reshape.shape[0], 'train samples')
print(quora_test_features_tf_reshape.shape[0], 'test samples')

quora_train_y_tf_cat = np_utils.to_categorical(quora_train_y_tf, 2)
quora_test_y_tf_cat = np_utils.to_categorical(quora_test_y_tf, 2)

# convert class vectors to binary class matrices for categorical cross_entropy
#y_train_kr = to_categorical(y_traincv)
#y_test_kr = to_categorical(y_testcv)

model = Sequential()
model.add(Dense(input_dim=quora_train_features_tf_reshape.shape[1], activation='relu', units=200))
model.add(Dropout(0.2))
model.add(Dense(input_dim=200, activation='relu', units=200))
model.add(Dropout(0.2))
model.add(Dense(input_dim=200, activation='relu', units=24)) #activation='softmax'
model.add(Dropout(0.2))
model.add(Dense(input_dim=24, kernel_initializer='uniform', activation='sigmoid', units=1)) #units=2 for cat.cr.ent
model.summary()


model.compile(loss='binary_crossentropy',
              #loss='categorical_crossentropy',
              #loss='mean_squared_error',
              optimizer=RMSprop(), 
              #optimizer=SGD(lr=0.001),
              metrics=['accuracy'])

history = model.fit(quora_train_features_tf_reshape, quora_train_y_tf_cat,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(quora_test_features_tf_reshape, quora_test_y_tf_cat))

score = model.evaluate(quora_test_features_tf_reshape, quora_test_y_tf_cat, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


predict_tf = model.predict_classes(quora_test_features_tf_reshape) #y_pred = np.around(model.predict(x_test_kr)[:,1])
print(log_loss(quora_test_y_tf_cat,predict_tf))


#### Vendo as 3 primeiras informações

In [ ]:
train.head(3)

In [ ]:
train.shape

#### Selecionando linhas

In [ ]:
train[4:6]

#### Selecionando Coluna

In [ ]:
train['question1'][200:205]

#### Selecionar duas colunas 

In [ ]:
train.loc[1:5,['question1','question2']]

#### Selecionando pela posição

In [ ]:
train.iloc[1:5,2:4]

#### Selecionando informação específica

In [ ]:
train[train.is_duplicate == 1].shape

#### Verificando por vazios

In [ ]:
vazioQ1 = []
vazioQ2 = []

for i in range(0,len(train.question2)):
    if type(train.question2[i]) is float:
        vazioQ2.append(i)
    if type(train.question1[i]) is float:
        vazioQ1.append(i)

In [ ]:
print(vazioQ1, vazioQ2)

#### Lower Case and removing punctuation

In [ ]:
def lower_rm_punct(phrase):
    if type(phrase) is not float:
        phrase = phrase.lower()
        phrase = phrase.translate(str.maketrans('','', string.punctuation))
    
    return phrase

In [ ]:
train.question1 = train.question1.apply(lambda x: lower_rm_punct(x))
train.question2 = train.question2.apply(lambda x: lower_rm_punct(x))

#### Function to Remove stopwords

In [ ]:
def rm_stopwords(phrase):
    stop = set(stopwords.words('english'))
    
    if type(phrase) is not float:
        phrase = [i for i in phrase.split() if i not in stop]
        phrase = ' '.join(phrase)
    return phrase

In [ ]:
train_stopw = train

train_stopw.question1 = train.question1.apply(lambda x: rm_stopwords(x))
train_stopw.question2 = train.question2.apply(lambda x: rm_stopwords(x))

#### Function to "stem"

#### From Stackoverflow:

##### At the very basics of it, the major difference between the porter and lancaster stemming algorithms is that the lancaster stemmer is significantly more aggressive than the porter stemmer. The three major stemming algorithms in use today are Porter, Snowball(Porter2), and Lancaster (Paice-Husk), with the aggressiveness continuum basically following along those same lines. Porter is the least aggressive algorithm, with the specifics of each algorithm actually being fairly lengthy and technical. Here is a break down for you though:

##### Porter: Most commonly used stemmer without a doubt, also one of the most gentle stemmers. One of the few stemmers that actually has Java support which is a plus, though it is also the most computationally intensive of the algorithms(Granted not by a very significant margin). It is also the oldest stemming algorithm by a large margin.

##### Porter2: Nearly universally regarded as an improvement over porter, and for good reason. Porter himself in fact admits that it is better than his original algorithm. Slightly faster computation time than porter, with a fairly large community around it.

##### Lancaster: Very aggressive stemming algorithm, sometimes to a fault. With porter and snowball, the stemmed representations are usually fairly intuitive to a reader, not so with Lancaster, as many shorter words will become totally obfuscated. The fastest algorithm here, and will reduce your working set of words hugely, but if you want more distinction, not the tool you would want.

##### Honestly, I feel that Snowball is usually the way to go. There are certain circumstances in which Lancaster will hugely trim down your working set, which can be very useful, however the marginal speed increase over snowball in my opinion is not worth the lack of precision. Porter has the most implementations though and so is usually the default go-to algorithm, but if you can, use snowball.


In [ ]:
def stem_lancaster(phrase):
    LS = LancasterStemmer()
    
    if type(phrase) is not float:
        phrase = [LS.stem(i) for i in phrase.split()]
        phrase = ' '.join(phrase)
    return phrase


#### Function to "lemmatize"

In [ ]:
def lemm_wordnet(phrase):
    lemm = WordNetLemmatizer()

    if type(phrase) is not float:
        phrase = [lemm.lemmatize(i) for i in phrase.split()]
        phrase = ' '.join(phrase)
    return phrase

#### Some Numbers

In [ ]:
def freq_all_text(question1,question2): #Pass all question

    question1_freq = []
    question2_freq = []
    for i in range(0,len(question1)):
        question1_freq += question1[i].split()
        question2_freq += str(question2[i]).split()
    
    len_question_freq = len(set(question1_freq + question2_freq))
    
    question1_freq = FreqDist(question1_freq)
    question2_freq = FreqDist(question2_freq)

    return question1_freq, question2_freq, len_question_freq

def percentage_by_text(phrase1, phrase2, question1_freq, question2_freq, len_question_freq):
    phrase2 = str(phrase2)
    
    for i in range(0, len(phrase1)):
            less_common_phrase1 = [i for i in set(phrase1.split()) if i not in phrase2.split()]
            less_common_phrase2 = [i for i in set(phrase2.split()) if i not in phrase1.split()]
            
    freq_less_common_phrase1 = 0
    freq_less_common_phrase2 = 0
            
    for i in less_common_phrase1:
        freq_less_common_phrase1 += question1_freq[i]
    
    freq_less_common_phrase1 = freq_less_common_phrase1/len_question_freq
    
    for i in less_common_phrase2:
        freq_less_common_phrase2 += question2_freq[i]
    
    freq_less_common_phrase2 = freq_less_common_phrase2/len_question_freq
    
    if freq_less_common_phrase1 == 0:
        return freq_less_common_phrase2
    
    elif freq_less_common_phrase2 == 0:
        return freq_less_common_phrase1
    
    elif freq_less_common_phrase1 >= freq_less_common_phrase2:
        return freq_less_common_phrase2
    
    else:
        return freq_less_common_phrase1

def percentages_by_phrase(phrase1,phrase2):
    phrase2 = str(phrase2)
    
    #Percentage between numbers of words
    if len(phrase1.split()) >= len(phrase2.split()):
        dif_wdsize = len(phrase2.split())/len(phrase1.split())
    else:
        dif_wdsize = len(phrase1.split())/len(phrase2.split())
   
    #Percentage between numbers of letters
    if len(phrase1) >= len(phrase2):
        dif_ltsize = len(phrase2)/len(phrase1)
    else:
        dif_ltsize = len(phrase1)/len(phrase2)
        
    #Percentage between letters and words per phrase
    dif_ltwdsize = (len(phrase1.split())/len(phrase1))/(len(phrase2.split())/len(phrase2))

    #Percentage of equal words
    equal = [i for i in set(phrase1.split()) if i in set(phrase2.split())]
    equal = ' '.join(equal)
    
    unique = set((phrase1 + ' ' + phrase2).split()) #Counting all unique words between the 2 phrases
    unique = ' '.join(unique)
    
    perc_equal_w = len(equal)/len(unique)
    
    return dif_wdsize, dif_ltsize, dif_ltwdsize, perc_equal_w

# Teste

In [ ]:
train = train[0:300]

#### Saving some new informations

In [ ]:
for i in range(0, len(train.question1)):
    p_size_word, p_size_letters, p_word_letters, p_equal_w = percentages_by_phrase(train.loc[i,'question1'],train.loc[i,'question2'])
    
    train.loc[i, 'p_size_word'] = p_size_word
    train.loc[i, 'p_size_letters'] = p_size_letters
    train.loc[i, 'p_word_letters'] = p_word_letters
    train.loc[i, 'p_equal_w'] = p_equal_w


In [ ]:
question1_freq, question2_freq, len_question_freq = freq_all_text(train.question1,train.question2)

In [ ]:
for i in range(0, len(train.question1)):
    
    p_less_common = percentage_by_text(train.loc[i,'question1'],train.loc[i,'question2'], question1_freq, question2_freq, len_question_freq)
    
    train.loc[i, 'p_less_common'] = p_less_common

#### Logistic Regression

In [ ]:
logit = sm.Logit(train['is_duplicate'], train.loc[:,['p_less_common','p_equal_w','p_word_letters']])

# fit the model
result = logit.fit()
result.summary()

In [ ]:
result.conf_int()

In [ ]:
result.predict(train.loc[:,['p_less_common','p_equal_w','p_word_letters']])